In [43]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import math
import pickle
import csv
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")

# 1 データ読み込み
- trainデータとtestデータ共に読み込んでマージ、不要な情報列を削除

In [44]:
df_tr = pd.read_csv('train_data.csv',index_col=0)
df_test = pd.read_csv("test_data.csv",index_col=0)
df_test['cover'] = 'a'
df = pd.concat([df_tr,df_test])
df.drop(columns=['Landsat_StartTime','YMD','PRODUCT_ID'],inplace=True)
df['year'] = df['year'].astype(int)
df = df.reset_index(drop=True)

# 2 データ前処理
## 2-1 クラスタリング
### 2-1-1 ラベルエンコーディング
- 'mesh20'をラベルエンコーディング実施

In [45]:
le = LabelEncoder()    
le.fit(df[['mesh20']])
list_label = sorted(list(set(le.classes_)))
map_label = {j:i for i,j in enumerate(list_label)}
dict_mesh = {}
dict_mesh['map_label'] = map_label
map_label = dict_mesh['map_label']
df['mesh20'] = df['mesh20'].map(map_label)

# ルール違反でした、、、
### 2-1-2 地理的に３つの区域（３クラス）にクラスタリング
- 経度、緯度情報のみを使用して３クラスに分類

In [46]:
clst = KMeans(n_clusters=3,
            init='k-means++',
            n_init=10,
            max_iter=300,
            random_state=0)
pred = clst.fit_predict(df[['lat','lon']])
df['cluster_id'] = pred

# ルール違反でした、、、
### 2-1-3 サブクラス作成
- 3つの区域（クラス）毎にサブクラスを３クラス作成_今回は全データを使ってクラスタリング

In [47]:
df_cls = df.copy()
df_cls.drop(columns=['cover','cluster_id'],inplace=True)
df_cls = df_cls.fillna(df_cls.median())
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_cls)
df_cls = pd.DataFrame(scaler.transform(df_cls),columns=df_cls.columns)
df_cls['cluster_id'] = df['cluster_id']
from sklearn.cluster import KMeans
clst = KMeans(n_clusters=3,
            init='k-means++',
            n_init=10,
            max_iter=300,
            random_state=0)
tmp_df = pd.DataFrame()
for i in range(3):
    pred = clst.fit_predict(df_cls[df_cls['cluster_id'] == i])
    tmp = pd.DataFrame({'id': df_cls[df_cls['cluster_id'] == i].index, 
                        'cls_id_2': pred})
    tmp_df = pd.concat([tmp_df,tmp])
tmp = tmp_df.sort_values('id').reset_index(drop=True)
df['cls_id_2'] = tmp['cls_id_2']

## 2-2 欠損値補完
- 主に画像データの欠損値を補完する

### 2-2-1 欠損値行の特定
   - 画像データのうち、50％以上が欠損値の行を補完する

In [48]:
# 行内のNaNの割合を計算する
nan_rows = df.iloc[:,85:3460].isna().sum(axis=1) / df.iloc[:,85:3460].shape[1]
# NaNの割合が50%以上の行を抽出する
high_nan_rows = df.iloc[:,85:3460][nan_rows >= 0.5]
nan_row_list = high_nan_rows.index

### 2-2-2 欠損値補完A
- 年毎のランドサットデータは前後1年のデータで補完する、2000年時は2001、2002のデータで補完

In [49]:
def fillna_with_column_values(arr, row, cols):
    for col in cols:
        if not np.isnan(arr[row, col]):
            return arr[row, col]
    return np.nan
arr = df.iloc[:,310:3460].to_numpy(dtype=float)
# 各列の欠損値を指定された条件で埋める
for j in range(3150):
    missing_rows = np.where(np.isnan(arr[:, j]))[0]
    for i in missing_rows:
        if j >= 0 and j < 150:
            cols = [j + 150, j + 300]
            cols = [col for col in cols if col >= 0 and col < arr.shape[1]]
            arr[i, j] = fillna_with_column_values(arr, i, cols)
        else:
            cols = [j - 150, j + 150]
            cols = [col for col in cols if col >= 0 and col < arr.shape[1]]
            arr[i, j] = fillna_with_column_values(arr, i, cols)
        
df.iloc[:,310:3460] = pd.DataFrame(arr, columns=df.iloc[:,310:3460].columns)

### 2-2-3 欠損値補完B
- Aで補完できなかった行を補完する
- 最寄りの５箇所を特定して、５箇所の平均値で補完する

#### 2-2-3-1 最寄りの５箇所を抽出
   - 緯度経度からターゲットから最も近い５箇所を抽出する（trainデータで補完する）

In [50]:
df_tr = df[df['cover'] != 'a']

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1_rad, lon1_rad = np.radians(lat1), np.radians(lon1)
    lat2_rad, lon2_rad = np.radians(lat2), np.radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

def closest_n_locations(df, index, n, exclude_indices):
    target_lat, target_lon = df.loc[index, 'lat'], df.loc[index, 'lon']
    distances = df.apply(lambda row: haversine(
        target_lat, target_lon, row['lat'], row['lon']), axis=1)
    distances = distances.drop(exclude_indices)
    closest_indices = distances.nsmallest(n+1).iloc[1:].index
    closest_rows = df.loc[closest_indices].copy()
    closest_rows['元の行番号'] = index  
    closest_rows['距離'] = distances[closest_indices].values  # 距離を追加
    return closest_rows

def closest_n_locations_for_indices(df, indices, n):
    result_df = pd.DataFrame()
    for index in indices:
        closest_n = closest_n_locations(df, index, n, indices)
        result_df = pd.concat([result_df, closest_n])
    return result_df.reset_index().rename(columns={'index': '抽出された行番号'})

# リスト内の各行番号に対して最も近い場所5箇所を抽出
closest_5_for_indices = closest_n_locations_for_indices(
    df_tr[['lat','lon']], nan_row_list, 5)

#### 2-2-3-2 ５箇所の平均値で欠損値を補完

In [51]:
tmp = closest_5_for_indices[['元の行番号','抽出された行番号']]
for original_index in tmp['元の行番号'].unique():
    closest_rows = tmp.loc[tmp['元の行番号'] == original_index, '抽出された行番号']
    mean_values = df.iloc[closest_rows,310:3460].mean()
    df.iloc[original_index,310:3460] = df.iloc[original_index,310:3460].fillna(
        mean_values)


### 2-2-4　欠損値補間C
- 時系列ランドサットデータの欠損値を補完する（測定年と同年の年ごとのランドセット画像MEDで補完する）
#### 2-2-4-1 補完対応表の作成
   - 時系列ランドサットデータと年毎のランドサット画像MEDの補完する際の列対応表を作成する

In [52]:
df_table = pd.read_csv('df_table_1.csv').reset_index(drop=True)
table_dict = {}
for index, row in df_table.iterrows():
    key = int(row['A'])
    value = row['B']    
    if pd.notna(value) and float(value).is_integer():
        value = int(value)
    elif pd.isna(value):
        value = float('nan')
    table_dict[key] = value

#### 2-2-4-2 欠損値補完（対応表に従って）

In [53]:
# M列の値に応じて欠損値を埋める関数
def fillna_based_on_m(row):
    m = row["year"]
    # M列の値が0から20の範囲内である場合のみ、欠損値を埋める処理を実行
    if 1999 <= m <= 2020:
        if m != 1999:
            start_col = 24
            end_col = 84
            offset = 150 * (m - 2000)

            for i in range(start_col, end_col):
                if np.isnan(row[i]):
                    if i in table_dict:
                        # 対応する列を計算し、値をコピー
                        source_col = table_dict[i-23] + 359 + offset
                        if not math.isnan(source_col):
                            row[i] = row[source_col]
                        else:
                            row[i] = float("nan")
                    else:
                        # 対応する列がない場合はNanに
                        row[i] = float("nan")
        else:
            start_col = 24
            end_col = 84
            offset = 0

            for i in range(start_col, end_col):
                if np.isnan(row[i]):
                    if i in table_dict:
                        # 対応する列を計算し、値をコピー
                        source_col = table_dict[i-23] + 359 + offset
                        if not math.isnan(source_col):
                            row[i] = row[source_col]
                        else:
                            row[i] = float("nan")
                    else:
                        # 対応する列がない場合はNanに
                        row[i] = float("nan")

    return row
# dfにfillna_based_on_m関数を適用
df = df.apply(fillna_based_on_m, axis=1)

## 2-3 特徴量作成
### 2-3-1 最寄りの'cover'を特徴量に追加
- 予測箇所から最も近い場所１０箇所を抽出して、予測箇所からの距離毎に特徴量に追加する
- 特徴量追加は、trainデータには、traiｎデータから、testデータにはtrain+testデータから追加する

#### 2-3-1-1 traiｎデータとtestデータに分割する

In [54]:
df_tr = df[df['cover'] != 'a']
df_ts = df[df['cover'] == 'a']
df_tr_list = df_tr.index
df_ts_list = df_ts.index


#### 2-3-1-2 最寄りの１０箇所を抽出する
- trainデータ用の１０箇所とtestデータ用の１０箇所それぞれ抽出する

In [55]:
def haversine_10(lat1, lon1, lat2, lon2):
    R = 6371
    lat1_rad, lon1_rad = np.radians(lat1), np.radians(lon1)
    lat2_rad, lon2_rad = np.radians(lat2), np.radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

def closest_10_locations(df, index, n, exclude_indices):
    target_lat, target_lon = df.loc[index, 'lat'], df.loc[index, 'lon']
    distances = df.apply(lambda row: haversine_10(
        target_lat, target_lon, row['lat'], row['lon']), axis=1)
    distances = distances.drop(exclude_indices)
    closest_indices = distances.nsmallest(n+1).iloc[1:].index
    closest_rows = df.loc[closest_indices].copy()
    closest_rows['元の行番号'] = index  
    closest_rows['距離'] = distances[closest_indices].values  # 距離を追加
    return closest_rows

def closest_10_locations_for_indices(df, indices,exclude_indices, n):
    result_df = pd.DataFrame()
    for index in indices:
        closest_n = closest_n_locations(df, index, n, exclude_indices)
        result_df = pd.concat([result_df, closest_n])
    return result_df.reset_index().rename(columns={'index': '抽出された行番号'})

In [56]:
# リスト内の各行番号に対して最も近い場所5箇所を抽出
closest_10_for_test = closest_10_locations_for_indices(
    df[['lat','lon']], df_ts_list, df_ts_list, 10)
closest_10_for_train = closest_10_locations_for_indices(
    df[['lat','lon']], df_tr_list, df_ts_list, 10)

#### 2-3-1-3 抽出した１０箇所を距離毎に分類して特徴量に加える
- trainデータ用testデータ用をマージして'cover'の値を取り出せるように加工
- ターゲットからの距離が100m未満は’cover_0'列へ100-200m以内は’cover_1'列へ、、、1km以上は'cover_10'列に追加するようにする

In [57]:
df_tmp_1 = pd.concat([closest_10_for_train,closest_10_for_test]).reset_index(drop=True)

In [58]:
grouped_tmp_1 = df_tmp_1.sort_values(['元の行番号','距離']).groupby(
    '元の行番号')['抽出された行番号'].apply(list).reset_index()
grouped_tmp_2 = df_tmp_1.sort_values(['元の行番号','距離']).groupby(
    '元の行番号')['距離'].apply(list).reset_index()
grouped = pd.concat([grouped_tmp_1,grouped_tmp_2['距離']],axis=1)

In [59]:
# df_tmp_3を作成
df_tmp_3 = pd.DataFrame()
for i, row in grouped.iterrows():
    extracted_row_numbers = row['抽出された行番号']
    distances = row['距離']
    
    distance_cover_dict = dict(zip(distances, extracted_row_numbers))
    
    for j in range(11):
        lower_bound = j * 0.1
        
        if j < 10:
            upper_bound = (j + 1) * 0.1
            cover_indices = [v for k, v in distance_cover_dict.items() if lower_bound <= k < upper_bound]
        else:
            cover_indices = [v for k, v in distance_cover_dict.items() if lower_bound <= k]
        
        if cover_indices:
            covers = df.loc[cover_indices, 'cover'].values
            df_tmp_3.loc[row['元の行番号'], f'cover_{j}'] = np.mean(covers, dtype=np.float64) 
        else:
            df_tmp_3.loc[row['元の行番号'], f'cover_{j}'] = np.nan

#### 2-3-1-4 上記の特徴量の統計値を特徴量に加える
- 平均、標準偏差、最大、最小、max-minを加える

In [60]:
df_tmp_3['mean'] = df_tmp_3.mean(axis=1)
df_tmp_3['max'] = df_tmp_3.max(axis=1)
df_tmp_3['min'] = df_tmp_3.min(axis=1)
df_tmp_3['max_min'] = df_tmp_3['max'] - df_tmp_3['min']
df_tmp_3['std'] = df_tmp_3.std(axis=1)

In [61]:
df = pd.concat([df,df_tmp_3],axis=1)

### 2-3-2 外れ値対応
- ４分位をとって75%-25%の10倍以上離れた値をNanに置き換え

In [62]:
def remove_outliers(cl_df):
    for column in cl_df.columns:
        # 列が数値データの場合のみ四分位範囲を計算
        if np.issubdtype(cl_df[column].dtypes, np.number):
            Q1 = cl_df[column].quantile(0.25)
            Q3 = cl_df[column].quantile(0.75)
            median = cl_df[column].median()
            IQR = Q3 - Q1 
            if IQR < median/10 :
                # 25%パーセンタイル値と75%パーセンタイル値が同じ場合、平均値の10倍以上を外れ値とする
                cl_df[column] = cl_df[column].apply(
                    lambda x: x if( median / 10 <= x <= median * 10 )else np.nan)                   
            else:
                # 外れ値の範囲を定義
                lower_bound = Q1 - 10 * IQR
                upper_bound = Q3 + 10 * IQR

                # 範囲内の値に制限
                cl_df[column] = cl_df[column].apply(
                    lambda x: x if lower_bound <= x <= upper_bound else np.nan)
    return cl_df
# 外れ値を取り除いたDataFrameを作成
df = remove_outliers(df)

### 2-3-3 海洋環境要因データを加工して特徴量に加える
- ’cover'に影響があると推測される下記特徴量を追加する
    - ’ef_cliff'を追加→'cliff_length'(海崖長)が０ではなく'coastal_dist'(海岸までの距離)が近いと影響があると仮説
    - ’ef_art'を追加→'aicial_length'(人工海岸線長）は埋立で'beach_length'(海浜長)’coast_length'と比較して値が大きく、かつ海岸線までの距離が近いと影響があると仮説
    - 'ef_bch'を追加→’biach_length'(海浜長）の比率が高く、かつ'coastal_dist'(海岸までの距離)が近いと影響があると仮説
    - 'ef_river'を追加→’river_area'(集水面積)が大きく'river_dist'(河口までの距離）が近いと影響があると仮説

In [63]:
df.loc[(df['cliff_length'].isnull()) & 
       (~df['aicial_length'].isnull() | 
        ~df['beach_length'].isnull()), 'cliff_length'] = 1
# df_tmp['ef_cliff'] = df['cliff_length']
df['ef_cliff'] = df.apply(
    lambda row: row['cliff_length'] * 2 
    if row['coastal_dist'] == 0 else row['cliff_length'] / row['coastal_dist'], axis=1)

df['ef_art'] = df['aicial_length'] \
/ (df['aicial_length'] + df['beach_length'])
df.loc[(df['ef_art'].isnull()) & (
    ~df['aicial_length'].isnull() | ~df['beach_length'].isnull()), 'ef_art'] = 0

df['ef_art'] = df.apply(
    lambda row: row['ef_art'] * 2 
    if row['coastal_dist'] == 0 else row['ef_art'] / row['coastal_dist'], axis=1)

df['ef_bch'] = df['beach_length'] / df['coast_length'].replace(0, float('inf'))
df['ef_bch'] = df['ef_bch'].replace(float('inf'), 0)
df['ef_bch'] = df['ef_bch'] * df['coast_length']
df['ef_bch'] = df.apply(
    lambda row: row['ef_bch'] * 2 
    if row['coastal_dist'] == 0 else row['ef_bch'] / row['coastal_dist'], axis=1)
df['ef_river'] = df['river_area'] / df['river_dist']

# 3 学習
## 3-1 学習用データ作成

In [66]:
df_train = df[df['cover'] != 'a']
df_train['cover'] = df_train['cover'].astype(float)
col_cat = ['mesh20','cluster_id','cls_id_2']
for col in col_cat:
    df_train[col] = df_train[col].astype("category")

In [67]:
X_train = df_train.drop(columns=['cover'])
y_train = df_train[["cover",'cluster_id','cls_id_2']]
id_train = pd.DataFrame(df_train.index)
id_train['cluster_id'] = df_train['cluster_id']
id_train['cls_id_2'] = df_train['cls_id_2']

## 3-2 学習
### 3-2-1 全データを一括学習

In [68]:
def train_lgb_1(input_X,
              input_y,
              input_id,
              params,
              list_nfold=[0,1,2,3,4],
              n_splits=5,
             ):
    train_oof = np.zeros(len(input_X))
    metrics = []
    imp = pd.DataFrame()
    input_y = input_y.drop(columns=['cluster_id','cls_id_2'])
    input_id = input_id.drop(columns=['cluster_id','cls_id_2'])
                             
    cv = list(KFold(n_splits, random_state=123, shuffle=True).split(input_X, input_y))
    for nfold in list_nfold:
        print("-"*20, nfold, "-"*20)
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        X_tr, y_tr, id_tr = input_X.loc[idx_tr, :], input_y.loc[idx_tr], input_id.loc[idx_tr, :]
        X_va, y_va, id_va = input_X.loc[idx_va, :], input_y.loc[idx_va], input_id.loc[idx_va, :]
        print(X_tr.shape, y_tr.shape)
        
        #train
        model = lgb.LGBMRegressor(**params)
        model.fit(X_tr,
                  y_tr,
                  eval_set=[(X_tr,y_tr), (X_va,y_va)],
                  early_stopping_rounds=50,
                  verbose=100,
                 )
        fname_lgb = "model_sbmt_1_6_lgb_fold{}.pickle".format(nfold)
        with open(fname_lgb, "wb") as f:
            pickle.dump(model, f, protocol=4)
        
        #evaluate
        y_tr_pred = model.predict(X_tr)
        y_va_pred = model.predict(X_va)
        metric_tr = np.sqrt(mean_squared_error(y_tr, y_tr_pred))
        metric_va = np.sqrt(mean_squared_error(y_va, y_va_pred))
        metrics.append([nfold, metric_tr, metric_va])
        print("[RMSE] tr: {:.4f}, va: {:.4f}".format(metric_tr, metric_va))
        
        # oof
        train_oof[idx_va] = y_va_pred
        
        # imp
        _imp = pd.DataFrame({"col": input_X.columns, 
                             "imp": model.feature_importances_,"nfold":nfold})
        imp = pd.concat([imp, _imp])
    
    # metric
    print("-"*20, "result", "-"*20)
    metrics = np.array(metrics)
    print(metrics)
    print("[cv] tr: {:.4f}+-{:.4f}, va: {:.4f}+-{:.4f}".format(
        metrics[:,1].mean(), metrics[:,1].std(),
        metrics[:,2].mean(), metrics[:,2].std(),
    ))
    print("[oof]{:.4f}".format(np.sqrt(mean_squared_error(input_y, train_oof))))
    
    # oof
    train_oof = pd.concat([input_id, pd.DataFrame({"pred" : train_oof})], axis=1)
    
    # importance
    imp = imp.groupby("col")["imp"].agg(["mean", "std"]).reset_index(drop=False)
    imp.columns = ["col", "imp", "imp_std"]
    
    return train_oof, imp, metrics     

In [69]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'metric': 'Root_Mean_Squared_Error',
    'learning_rate': 0.05,
    'num_leaves': 32,
    'subsample': 0.7,
    'subsample_freq': 1,
    'feature_fraction': 0.8,
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 50,
    'n_estimators': 1000,
    'random_state': 123,
    'importance_type': 'gain',
}

train_oof, imp, metrics = train_lgb_1(X_train,
                                    y_train,
                                    id_train,
                                    params,
                                    list_nfold=[0,1,2,3,4],
                                    n_splits=5,
                                   )

-------------------- 0 --------------------
(11312, 3482) (11312, 1)
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=50, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=50
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.0799263	valid_1's rmse: 0.116828
[200]	training's rmse: 0.0642602	valid_1's rmse: 0.114661
[300]	training's rmse: 0.0538535	valid_1's rmse: 0.112897
[400]	training's rmse: 0.0464674	valid_1's rmse: 0.111868
[500]	training's rmse: 0.0410587	valid_1's rmse: 0.111289
[600]	training's rmse: 0.0368198	valid_1's rmse: 0.110879
[700]	training's rmse: 0.0332483	valid_1's rmse: 0.11051
[800]	training's rmse: 0.0304542	valid_1's rmse: 0.110225
[900]	training's rmse: 

In [70]:
imp.sort_values('imp',ascending=False).head(30)

,col,imp,imp_std
3442,cover_0,3162.108555,22.949851
3470,min,554.126198,37.535101
3468,mean,490.754809,39.652152
3477,std,152.779309,27.562692
3466,max,55.513771,8.814508
3445,cover_2,13.557671,2.229298
3443,cover_1,10.553051,3.505403
2739,MIN_GARI,9.227729,2.863154
3467,max_min,6.965922,0.729239
3471,month,6.924054,0.679165


### 3-2-2 地理的な３つの区域毎に学習

In [71]:
def train_lgb_3(input_X,
              input_y,
              input_id,
              params,
              list_nfold=[0, 1, 2, 3, 4],
              n_splits=5,
              ):
    metrics_list = []
    train_oof_list =[]
    imp_list =[]
    for i in range(3):
            tmp_X = input_X[input_X['cluster_id'] == i]
            tmp_y = input_y[input_y['cluster_id'] == i]
            tmp_id = input_id[input_id['cluster_id'] == i]

            tmp_y.drop(columns=['cluster_id','cls_id_2'], inplace=True)
            tmp_id.drop(columns=['cluster_id','cls_id_2'], inplace=True)
            tmp_X.reset_index(drop=True, inplace=True)
            tmp_y.reset_index(drop=True, inplace=True)
            tmp_id.reset_index(drop=True, inplace=True)

            train_oof = np.zeros(len(tmp_X))
            metrics = []
            imp = pd.DataFrame()

                # cross-validation
            cv = list(KFold(n_splits, random_state=123, shuffle=True).split(
                    tmp_X, tmp_y))

            for nfold in list_nfold:
                print("-" * 20, nfold, "-" * 20)
                idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
                X_tr, y_tr, id_tr = tmp_X.loc[idx_tr, :], tmp_y.loc[idx_tr], \
                                    tmp_id.loc[idx_tr, :]
                X_va, y_va, id_va = tmp_X.loc[idx_va, :], tmp_y.loc[idx_va], \
                                    tmp_id.loc[idx_va, :]
                print(X_tr.shape, y_tr.shape)

                # train
                model = lgb.LGBMRegressor(**params)
                model.fit(X_tr,
                          y_tr,
                          eval_set=[(X_tr, y_tr), (X_va, y_va)],
                          early_stopping_rounds=50,
                          verbose=100,
                         )
                fname_lgb = "model_sbmt_1_6_cls{0}_lgb_fold{1}.pickle".format(i, nfold)
                with open(fname_lgb, "wb") as f:
                    pickle.dump(model, f, protocol=4)
                    
                # evaluate
                y_tr_pred = model.predict(X_tr)
                y_va_pred = model.predict(X_va)
                metric_tr = np.sqrt(mean_squared_error(y_tr, y_tr_pred))
                metric_va = np.sqrt(mean_squared_error(y_va, y_va_pred))
                metrics.append([nfold, metric_tr, metric_va])
                print("[RMSE] tr: {:.4f}, va: {:.4f}".format(metric_tr, metric_va))

                # oof
                train_oof[idx_va] = y_va_pred

                # imp
                _imp = pd.DataFrame(
                    {"col": tmp_X.columns, "imp": model.feature_importances_, 
                     "nfold": nfold})
                imp = pd.concat([imp, _imp])

            # metric
            print("-" * 20, "cls{}_result".format(i), "-" * 20)
            metrics = np.array(metrics)
            print(metrics)
            print("[cv] tr: {:.4f}+-{:.4f}, va: {:.4f}+-{:.4f}".format(
                metrics[:, 1].mean(), metrics[:, 1].std(),
                metrics[:, 2].mean(), metrics[:, 2].std(),
                ))
            print("[oof]{:.4f}".format(np.sqrt(mean_squared_error(tmp_y, train_oof))))
            metrics_list.append(metrics)

            # oof
            train_oof = pd.concat([tmp_id, pd.DataFrame({"pred": train_oof})], axis=1)
            train_oof_list.append(train_oof)

            # importance
            imp = imp.groupby("col")["imp"].agg(["mean", "std"]).reset_index(drop=False)
            imp.columns = ["col", "imp", "imp_std"]
            imp_list.append(imp)

    return train_oof_list, imp_list, metrics_list

In [72]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'metric': 'Root_Mean_Squared_Error',
    'learning_rate': 0.05,
    'num_leaves': 32,
    'subsample': 0.7,
    'subsample_freq': 1,
    'feature_fraction': 0.8,
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 50,
    'n_estimators': 1000,
    'random_state': 123,
    'importance_type': 'gain',
}

train_oof, imp, metrics = train_lgb_3(X_train,
                                    y_train,
                                    id_train,
                                    params,
                                    list_nfold=[0,1,2,3,4],
                                    n_splits=5,
                                   )

-------------------- 0 --------------------
(4077, 3482) (4077, 1)
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=50, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=50
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.0775861	valid_1's rmse: 0.112777
[200]	training's rmse: 0.0539212	valid_1's rmse: 0.112039
[300]	training's rmse: 0.0407875	valid_1's rmse: 0.111551
Early stopping, best iteration is:
[258]	training's rmse: 0.0454688	valid_1's rmse: 0.111342
[RMSE] tr: 0.0455, va: 0.1113
-------------------- 1 --------------------
(4077, 3482) (4077, 1)
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[L

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.0739279	valid_1's rmse: 0.104521
[200]	training's rmse: 0.0551448	valid_1's rmse: 0.102013
[300]	training's rmse: 0.0442129	valid_1's rmse: 0.100724
[400]	training's rmse: 0.0369986	valid_1's rmse: 0.10003
[500]	training's rmse: 0.031874	valid_1's rmse: 0.0993602
[600]	training's rmse: 0.0281274	valid_1's rmse: 0.0991129
Early stopping, best iteration is:
[592]	training's rmse: 0.0283849	valid_1's rmse: 0.0990345
[RMSE] tr: 0.0284, va: 0.0990
-------------------- 4 --------------------
(5163, 3482) (5163, 1)
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=50, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=50
Training until v

In [73]:
metrics

[array([[0.        , 0.04546878, 0.11134173],
        [1.        , 0.03237271, 0.12620493],
        [2.        , 0.02224194, 0.12840409],
        [3.        , 0.04017336, 0.11528105],
        [4.        , 0.01627479, 0.12463069]]),
 array([[0.        , 0.02270733, 0.10098956],
        [1.        , 0.03709703, 0.10318531],
        [2.        , 0.02390023, 0.11292029],
        [3.        , 0.02838491, 0.09903454],
        [4.        , 0.02121805, 0.10564355]]),
 array([[0.        , 0.04215817, 0.07973951],
        [1.        , 0.02968822, 0.10307291],
        [2.        , 0.07553931, 0.08576264],
        [3.        , 0.03717035, 0.08679931],
        [4.        , 0.02681353, 0.08719117]])]

In [74]:
imp[2].sort_values('imp',ascending=False)[:30]

,col,imp,imp_std
3442,cover_0,188.670119,43.606909
3470,min,79.169690,47.593659
3468,mean,55.448515,34.489858
3477,std,28.758034,11.427822
3466,max,6.084168,4.076325
528,MAX_GEMI_2017,3.023788,1.074437
1042,MAX_TIRS1_2014,2.451206,3.931257
3443,cover_1,2.441550,0.833542
3356,MIN_VARIgreen_2014,1.860930,1.093803
2803,MIN_GLI_2018,1.736956,1.426598


### 3-2-3 地理的区域毎、かつサブクラス毎に学習

In [75]:
def train_lgb_9(input_X,
              input_y,
              input_id,
              params,
              list_nfold=[0, 1, 2, 3, 4],
              n_splits=5,
              cls_num=3,
              cls_num_2=3,
              ):
    metrics_list = []
    train_oof_list =[]
    imp_list =[]
    for i in range(cls_num):
        tmp_2_X = input_X[input_X['cluster_id'] == i]
        tmp_2_y = input_y[input_y['cluster_id'] == i]
        tmp_2_id = input_id[input_id['cluster_id'] == i]
        
        for j in range(cls_num_2):
                tmp_X = tmp_2_X[tmp_2_X['cls_id_2'] == j]
                tmp_y = tmp_2_y[tmp_2_y['cls_id_2'] == j]
                tmp_id = tmp_2_id[tmp_2_id['cls_id_2'] == j]
            
                tmp_y.drop(columns=['cluster_id','cls_id_2'], inplace=True)
                tmp_id.drop(columns=['cluster_id','cls_id_2'], inplace=True)
                tmp_X.reset_index(drop=True, inplace=True)
                tmp_y.reset_index(drop=True, inplace=True)
                tmp_id.reset_index(drop=True, inplace=True)

                train_oof = np.zeros(len(tmp_X))
                metrics = []
                imp = pd.DataFrame()

                # cross-validation
                cv = list(KFold(n_splits, random_state=123, shuffle=True).split(
                    tmp_X, tmp_y))

                for nfold in list_nfold:
                    print("-" * 20, nfold, "-" * 20)
                    idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
                    X_tr, y_tr, id_tr = tmp_X.loc[idx_tr, :], tmp_y.loc[idx_tr], \
                                        tmp_id.loc[idx_tr, :]
                    X_va, y_va, id_va = tmp_X.loc[idx_va, :], tmp_y.loc[idx_va], \
                                        tmp_id.loc[idx_va, :]
                    print(X_tr.shape, y_tr.shape)

                    # train
                    model = lgb.LGBMRegressor(**params)
                    model.fit(X_tr,
                              y_tr,
                              eval_set=[(X_tr, y_tr), (X_va, y_va)],
                              early_stopping_rounds=50,
                              verbose=100,
                              )
                    fname_lgb = "model_sbmt_1_6_cls{0}_{1}_lgb_fold{2}.pickle".format(
                        i,j, nfold)
                    with open(fname_lgb, "wb") as f:
                        pickle.dump(model, f, protocol=4)

                    # evaluate
                    y_tr_pred = model.predict(X_tr)
                    y_va_pred = model.predict(X_va)
                    metric_tr = np.sqrt(mean_squared_error(y_tr, y_tr_pred))
                    metric_va = np.sqrt(mean_squared_error(y_va, y_va_pred))
                    metrics.append([nfold, metric_tr, metric_va])
                    print("[RMSE] tr: {:.4f}, va: {:.4f}".format(metric_tr, metric_va))

                    # oof
                    train_oof[idx_va] = y_va_pred

                    # imp
                    _imp = pd.DataFrame(
                        {"col": tmp_X.columns, 
                         "imp": model.feature_importances_, 
                         "nfold": nfold})
                    imp = pd.concat([imp, _imp])

                # metric
                print("-" * 20, "cls{}_{}_result".format(i,j), "-" * 20)
                print(fname_lgb)
                metrics = np.array(metrics)
                print(metrics)
                print("[cv] tr: {:.4f}+-{:.4f}, va: {:.4f}+-{:.4f}".format(
                    metrics[:, 1].mean(), metrics[:, 1].std(),
                    metrics[:, 2].mean(), metrics[:, 2].std(),
                ))
                print("[oof]{:.4f}".format(np.sqrt(mean_squared_error(tmp_y, train_oof))))
                metrics_list.append(metrics)

                # oof
                train_oof = pd.concat([tmp_id, pd.DataFrame({"pred": train_oof})], axis=1)
                train_oof_list.append(train_oof)

                # importance
                imp = imp.groupby("col")["imp"].agg(["mean", "std"]).reset_index(drop=False)
                imp.columns = ["col", "imp", "imp_std"]
                imp_list.append(imp)        

    return train_oof_list, imp_list, metrics_list

In [76]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'metric': 'Root_Mean_Squared_Error',
    'learning_rate': 0.05,
    'num_leaves': 32,
    'subsample': 0.7,
    'subsample_freq': 1,
    'feature_fraction': 0.8,
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 50,
    'n_estimators': 1000,
    'random_state': 123,
    'importance_type': 'gain',
}

train_oof_list, imp_list, metrics_list = train_lgb_9(X_train,
                                                   y_train,
                                                   id_train,
                                                   params,
                                                   list_nfold=[0,1,2,3,4],
                                                   n_splits=5,
                                                   cls_num=3,
                                                   cls_num_2=3,
                                                  )

-------------------- 0 --------------------
(1741, 3482) (1741, 1)
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=50, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=50
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.0577995	valid_1's rmse: 0.0853211
[200]	training's rmse: 0.0430167	valid_1's rmse: 0.0834213
[300]	training's rmse: 0.0338167	valid_1's rmse: 0.0827284
Early stopping, best iteration is:
[333]	training's rmse: 0.0314572	valid_1's rmse: 0.0822579
[RMSE] tr: 0.0315, va: 0.0823
-------------------- 1 --------------------
(1741, 3482) (1741, 1)
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5

[100]	training's rmse: 0.0844656	valid_1's rmse: 0.119983
[200]	training's rmse: 0.0595386	valid_1's rmse: 0.115405
[300]	training's rmse: 0.0452109	valid_1's rmse: 0.114284
Early stopping, best iteration is:
[330]	training's rmse: 0.0419046	valid_1's rmse: 0.113773
[RMSE] tr: 0.0419, va: 0.1138
-------------------- cls0_1_result --------------------
model_sbmt_1_6_cls0_1_lgb_fold4.pickle
[[0.         0.03579707 0.11440256]
 [1.         0.03711991 0.12577636]
 [2.         0.08850061 0.14106388]
 [3.         0.02761745 0.11888153]
 [4.         0.04190457 0.11377251]]
[cv] tr: 0.0462+-0.0217, va: 0.1228+-0.0101
[oof]0.1232
-------------------- 0 --------------------
(928, 3482) (928, 1)
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=50, min_

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.112334	valid_1's rmse: 0.134462
[200]	training's rmse: 0.0914533	valid_1's rmse: 0.129877
[300]	training's rmse: 0.0783979	valid_1's rmse: 0.129701
Early stopping, best iteration is:
[274]	training's rmse: 0.0811782	valid_1's rmse: 0.129023
[RMSE] tr: 0.0812, va: 0.1290
-------------------- cls1_0_result --------------------
model_sbmt_1_6_cls1_0_lgb_fold4.pickle
[[0.         0.06197422 0.15577918]
 [1.         0.0879376  0.14565119]
 [2.         0.08400462 0.15381696]
 [3.         0.0811424  0.14234665]
 [4.         0.08117818 0.12902339]]
[cv] tr: 0.0792+-0.0090, va: 0.1453+-0.0096
[oof]0.1456
-------------------- 0 --------------------
(2630, 3482) (2630, 1)
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.

[100]	training's rmse: 0.0913081	valid_1's rmse: 0.140317
[200]	training's rmse: 0.0645193	valid_1's rmse: 0.138133
[300]	training's rmse: 0.0507842	valid_1's rmse: 0.137478
[400]	training's rmse: 0.0421459	valid_1's rmse: 0.136189
[500]	training's rmse: 0.0364508	valid_1's rmse: 0.135452
Early stopping, best iteration is:
[539]	training's rmse: 0.0348172	valid_1's rmse: 0.135146
[RMSE] tr: 0.0348, va: 0.1351
-------------------- 4 --------------------
(2100, 3482) (2100, 1)
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=50, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=50
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.0904772	valid_1's rmse: 0.145458
[200]	training's 

Early stopping, best iteration is:
[1]	training's rmse: 0.130323	valid_1's rmse: 0.138571
[RMSE] tr: 0.0900, va: 0.0443
-------------------- cls2_1_result --------------------
model_sbmt_1_6_cls2_1_lgb_fold4.pickle
[[0.         0.06501197 0.12895811]
 [1.         0.07379598 0.10990792]
 [2.         0.09066061 0.03367259]
 [3.         0.08996859 0.04428572]
 [4.         0.08996859 0.04428572]]
[cv] tr: 0.0819+-0.0106, va: 0.0722+-0.0392
[oof]0.0875
-------------------- 0 --------------------
(260, 3482) (260, 1)
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=50, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=50
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.147718	valid_

In [77]:
metrics_list

[array([[0.        , 0.03145719, 0.08225792],
        [1.        , 0.05060498, 0.06564522],
        [2.        , 0.04048305, 0.06883107],
        [3.        , 0.04444071, 0.08169995],
        [4.        , 0.02448431, 0.06543654]]),
 array([[0.        , 0.03579707, 0.11440256],
        [1.        , 0.03711991, 0.12577636],
        [2.        , 0.08850061, 0.14106388],
        [3.        , 0.02761745, 0.11888153],
        [4.        , 0.04190457, 0.11377251]]),
 array([[0.        , 0.05871604, 0.17644928],
        [1.        , 0.10608069, 0.1836682 ],
        [2.        , 0.06916512, 0.17088142],
        [3.        , 0.04247962, 0.16201819],
        [4.        , 0.11211039, 0.19180061]]),
 array([[0.        , 0.06197422, 0.15577918],
        [1.        , 0.0879376 , 0.14565119],
        [2.        , 0.08400462, 0.15381696],
        [3.        , 0.0811424 , 0.14234665],
        [4.        , 0.08117818, 0.12902339]]),
 array([[0.        , 0.03129784, 0.06262796],
        [1.        , 0.049

In [78]:
imp_list[8].sort_values('imp',ascending=False).head(30)

,col,imp,imp_std
3477,std,48.580179,30.636281
3442,cover_0,39.826205,19.962763
3470,min,34.518679,18.504829
3466,max,16.382101,13.280762
2799,MIN_GLI_2014,8.650922,6.469825
2907,MIN_IF_2014,6.649405,5.399920
2474,MIN_CVI_2016,5.732517,7.164911
1350,MED_CVI_2016,5.404938,5.085990
2238,MED_WVP,3.336001,2.261879
1676,MED_GLI_2016,2.566310,2.457506


# 4 予測
## 4-1 予測用データ作成

In [79]:
df_test = df[df['cover'] == 'a'].reset_index(drop=True)
df_test = df_test.drop(columns=['cover'])
col_cat = ['mesh20','cluster_id','cls_id_2']
for col in col_cat:
    df_test[col] = df_test[col].astype("category")

In [80]:
X_test = df_test.copy()
id_test = pd.DataFrame(df_test.index)
id_test['cluster_id'] = df_test['cluster_id']
id_test['cls_id_2'] = df_test['cls_id_2']

## 4-2 予測
- ３つの学習モデルをアンサンブル実施。重みづけは実施せず、平均値を予測値とした

In [81]:
def predict_lgb_9(input_X,
                input_id,
                list_nfold_0=[0,1,2,3,4],
                list_nfold_1=[0,1,2,3,4],
                list_nfold_2=[0,1,2,3,4],
               ):
    pred_list = []
    for i in range(3):
            tmp_2_X = input_X[input_X['cluster_id'] == i]
            tmp_2_id = input_id[input_id['cluster_id'] == i]
            tmp_2_id.drop(columns=['cluster_id'], inplace=True)
            
            for j in range(3):                 
                    tmp_X = tmp_2_X[tmp_2_X['cls_id_2'] == j]
                    tmp_id = tmp_2_id[tmp_2_id['cls_id_2'] == j]
                    tmp_id.drop(columns=['cls_id_2'], inplace=True)
                    pred = np.zeros((len(tmp_X), 
                                     len(list_nfold_0)
                                    + len(list_nfold_1) 
                                     + len(list_nfold_2)))
                    
                    for nfold in list_nfold_2:
                        print("-"*20, nfold, "-"*20)
                        fname_lgb_1 = "model_sbmt_1_6_cls{}_lgb_fold{}.pickle".format(
                            i,nfold)
                        with open(fname_lgb_1, "rb") as f:
                            model = pickle.load(f)
                        pred[:, len(list_nfold_0) + 
                             len(list_nfold_1) + nfold] = model.predict(tmp_X)
                                            
                    for nfold in list_nfold_1:
                        print("-"*20, nfold, "-"*20)
                        fname_lgb_2 = "model_sbmt_1_6_cls{}_{}_lgb_fold{}.pickle".format(
                            i,j,nfold)
                        with open(fname_lgb_2, "rb") as f:
                            model = pickle.load(f)
                        pred[:, len(list_nfold_0) + nfold] = model.predict(tmp_X)
                    
                    for nfold in list_nfold_0:
                        print("-"*20, nfold, "-"*20)
                        fname_lgb_3 = "model_sbmt_1_6_lgb_fold{}.pickle".format(nfold)
                        with open(fname_lgb_3, "rb") as f:
                            model = pickle.load(f)
                        pred[:, nfold] = model.predict(tmp_X)
                    
                    df_pred = pd.DataFrame({"pred": pred.mean(axis=1)})
                    df_pred.index = tmp_id.index
                    pred = pd.concat([
                        tmp_id,df_pred,], axis=1)
                    pred_list.append(pred)
                    text = 'cls{}_{}_Done.'.format(i,j)
                    print(len(tmp_X))
                    print(text + '_' + fname_lgb_3 + '_' + fname_lgb_1 + '_' + fname_lgb_2)
                    print(len(pred))                    

    return pred_list

In [82]:
test_pred_2 = predict_lgb_9(
    X_test,id_test,list_nfold_1=[0,1,2,3,4],list_nfold_2=[0,1,2,3,4],
)

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
709
cls0_0_Done._model_sbmt_1_6_lgb_fold4.pickle_model_sbmt_1_6_cls0_lgb_fold4.pickle_model_sbmt_1_6_cls0_0_lgb_fold4.pickle
709
-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 ------------

In [83]:
test_pred = pd.DataFrame()
for i in range(9):
    tmp = test_pred_2[i]
    test_pred = pd.concat([test_pred,tmp])
test_pred = test_pred.sort_values(0)
# 負の値はゼロに、１を超えた値は１に修正
test_pred['pred'] = test_pred['pred'].apply(lambda x: 0 if x<0 else (1 if x>1 else x))

In [84]:
test_pred.to_csv("sbmt_test_1_6.csv", header=False,index=None)